# Inference model monitoring
In this notebook we will have a look at how to set up monitoring hooks for your inference models. The Geti SDK provides several basic triggers and actions that can be used to construct pipelines for, for instance, data collection, alerting, or other actions that need to take place based on inference results. 

These pipelines are referred to as `post inference hooks` and can be added to any `Deployment` for any project. In this notebook we will create a post inference hook that implements the following:

For every inferred frame or image, check if any of the predicted objects/labels have a low model confidence (i.e. probability < 55 %). If so, send the image to the Geti server. The image will be stored in a new dataset called `Inferred images`, within the original project.

In [ ]:
# As usual we will connect to the platform first, using the server details from the .env file

from geti_sdk import Geti
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()

geti = Geti(server_config=geti_server_configuration)

## Selecting a project


In [ ]:
PROJECT_NAME = "COCO animal detection demo"
project = geti.get_project(PROJECT_NAME)

## Create deployment for the project

In [ ]:
deployment = geti.deploy_project(PROJECT_NAME)

## Checking deployment output

In [ ]:
deployment.load_inference_models()

In [ ]:
import cv2

from geti_sdk.demos import EXAMPLE_IMAGE_PATH
from geti_sdk.utils import show_image_with_annotation_scene

numpy_image = cv2.imread(EXAMPLE_IMAGE_PATH)
numpy_rgb = cv2.cvtColor(numpy_image, cv2.COLOR_BGR2RGB)

prediction = deployment.infer(numpy_rgb)

show_image_with_annotation_scene(numpy_rgb, prediction, show_in_notebook=True);

## Configuring the post inference hook

In [ ]:
from geti_sdk.post_inference_hooks import (
    ConfidenceTrigger,
    GetiDataCollection,
    PostInferenceHook,
)

trigger = ConfidenceTrigger(threshold=0.9)
action = GetiDataCollection(
    session=geti.session,
    workspace_id=geti.workspace_id,
    project=project,
    dataset="Inferred images",
)
hook = PostInferenceHook(trigger=trigger, action=action, max_threads=2)

In [ ]:
deployment.add_post_inference_hook(hook)

In [ ]:
prediction = deployment.infer(numpy_rgb)

In [ ]:
from geti_sdk.demos import get_person_car_bike_video

video_path = get_person_car_bike_video()